## IMPORT LIBRARIES

In [1]:
# Importing Libraries

# Data Wrangling
import numpy as np
import pandas as pd

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Filter Warning
import warnings
warnings.filterwarnings("ignore")

## OVERVIEW

In [2]:
# Load data
df = pd.read_csv("data/rfm_data.csv")

In [3]:
# Check info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CustomerID          1000 non-null   int64  
 1   PurchaseDate        1000 non-null   object 
 2   TransactionAmount   1000 non-null   float64
 3   ProductInformation  1000 non-null   object 
 4   OrderID             1000 non-null   int64  
 5   Location            1000 non-null   object 
dtypes: float64(1), int64(2), object(3)
memory usage: 47.0+ KB


## PREPROCESSING 

### GET RFM VALUES

In [4]:
# Check Last date
max_date = pd.to_datetime(df["PurchaseDate"].max())

In [5]:
# Insert to dataframe
df["LastPurchase"] = max_date

In [7]:
# Change to datetime
df["PurchaseDate"] = pd.to_datetime(df["PurchaseDate"])

In [12]:
# Get Recency
df["Recency"] = df["LastPurchase"] - df["PurchaseDate"]
df["Recency"] = df["Recency"].dt.days

In [18]:
# Get Frequency
freq = df.groupby("CustomerID", as_index=False)["OrderID"].count()
freq = freq.rename(columns={"OrderID":"Frequency"})

In [20]:
# Merge Frequency vals to original df
df_new = pd.merge(df, freq, on ="CustomerID")

In [22]:
# Get Monetary
mone = df.groupby("CustomerID", as_index=False)["TransactionAmount"].sum()
mone = mone.rename(columns={"TransactionAmount":"Monetary"})

In [23]:
# Merge Monetary vals to original df
df_rfm = pd.merge(df_new, mone, on ="CustomerID")

In [24]:
# Show rfm data
df_rfm.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,LastPurchase,Recency,Frequency,Monetary
0,8814,2023-04-11,943.31,Product C,890075,Tokyo,2023-06-10,60,1,943.31
1,2188,2023-04-11,463.70,Product A,176819,London,2023-06-10,60,1,463.70
2,4608,2023-04-11,80.28,Product A,340062,New York,2023-06-10,60,1,80.28
3,2559,2023-04-11,221.29,Product A,239145,London,2023-06-10,60,1,221.29
4,9482,2023-04-11,739.56,Product A,194545,Paris,2023-06-10,60,1,739.56


In [35]:
# export to csv
df_rfm.to_csv("df_rfm_cleaned.csv")

#### GET RFM SCORE

In [33]:
# check quantile score
df_rfm[["Recency", "Frequency", "Monetary"]].quantile([0.25, 0.5, 0.75])

,Recency,Frequency,Monetary
0.25,15.0,1.0,283.9250
0.50,32.0,1.0,566.7100
0.75,45.0,1.0,805.3725


In [37]:
# check quantile score
df_rfm[["Recency", "Frequency", "Monetary"]].quantile([0.2, 0.4, 0.6, 0.8])

,Recency,Frequency,Monetary
0.2,12.0,1.0,224.450
0.4,25.0,1.0,456.494
0.6,37.0,1.0,668.238
0.8,47.2,1.0,848.916


![](https://blog.rsquaredacademy.com/img/rfm_segments_table.png)

![](https://runawayhorse001.github.io/LearningApacheSpark/_images/rfm_business.png)